# This ipynd tabulates the study credits for study modules from Aalto University in ascending order

In [1]:
import pandas as pd
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# Initializing webdriver and web page

In [2]:
option = webdriver.ChromeOptions()
option.add_argument("start-minimized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=option)
driver.get('https://www.aalto.fi/fi/sivuaineet#/')

In [3]:
driver.find_element(By.XPATH, "//button[text()='Kiellä kaikki']").click()

# Finding the Finnish module links

In [4]:
parent_elements = driver.find_elements(By.CLASS_NAME, "hub-listing__results__group")

In [5]:
links_finnish = []
for element in parent_elements:
    child_elements = element.find_elements(By.XPATH, ".//*")
    links_finnish.append(child_elements[0].get_attribute("href"))

In [6]:
print(f"MODULE LINK COUNT: {len(links_finnish)}")

MODULE LINK COUNT: 191


# Fetch module name and credits for each Finnish module

In [7]:
modules_finnish = []
for link in links_finnish:
    driver.get(link)
    module_name = driver.find_element(By.XPATH, "//h1[contains(text(), *)]").text
    try:
        find_credits = driver.find_element(By.XPATH, "//*[contains(text(), 'ECTS')]")
        module_credits = find_credits.text
    except:
        module_credits = None
        
    modules_finnish.append({"Module name": module_name, "ECTS": module_credits})

In [8]:
modules_finnish = [module for module in modules_finnish if module["ECTS"] is not None]

In [9]:
print(f"FINNISH MODULE COUNT: {len(modules_finnish)}")

FINNISH MODULE COUNT: 69


# Doing the same but for modules in English

In [10]:
driver.get("https://www.aalto.fi/en/minors#/")

In [11]:
parent_elements = driver.find_elements(By.CLASS_NAME, "hub-listing__results__group")

In [12]:
links_english = []
for element in parent_elements:
    child_elements = element.find_elements(By.XPATH, ".//*")
    links_english.append(child_elements[0].get_attribute("href"))

In [13]:
print(f"MODULE LINK COUNT: {len(links_english)}")

MODULE LINK COUNT: 191


In [14]:
modules_english = []
for link in links_english:
    driver.get(link)
    module_name = driver.find_element(By.XPATH, "//h1[contains(text(), *)]").text
    try:
        find_credits = driver.find_element(By.XPATH, "//*[contains(text(), 'ECTS')]")
        module_credits = find_credits.text
    except:
        module_credits = None
        
    modules_english.append({"Module name": module_name, "ECTS": module_credits})

In [15]:
modules_english = [module for module in modules_english if module["ECTS"] is not None]

In [16]:
print(f"ENGLISH MODULE COUNT: {len(modules_english)}")

ENGLISH MODULE COUNT: 147


# Sort modules by second item 

In [17]:
modules = modules_finnish + modules_english

In [18]:
sorted_modules = sorted(modules, key=lambda module: module["ECTS"])

# Converting to pandas dataframe

In [19]:
modules_df = pd.DataFrame(sorted_modules)

In [20]:
modules_df

,Module name,ECTS
0,Biomass Refining (minor),
1,MEDes - Master of European Design,120 ECTS
2,Nordic Visual Studies and Art Education,15 ECTS
3,Arkkitehtuurin historia ja teoria (sivuaine),15–25 ECTS
4,Elokuvaleikkaus,15–25 ECTS
...,...,...
211,"International minor, only for ENG master students",25–30 ECTS
212,Valtakunnallinen liiketoimintaosaamisen sivuai...,25–40 ECTS
213,Innovation and Entrepreneurship for Urban Mobi...,30 ECTS
214,ME310 Aalto - Global Innovation Program,30 ECTS


# Creating an Excel spreadsheet

In [24]:
modules_df.to_excel('example.xlsx', index=True)